# Lab 3.1 - Główne składowe selfie

**Wykonanie rozwiązań: Marcin Przewięźlikowski**

https://github.com/mprzewie/ml_basics_course

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.io import loadmat
import matplotlib
import os

## Dataset

Robimy sobie i bliskim/współlokatorkom/współlokatorom/innym znajomym 15+ (ale im więcej tym lepiej) różnych selfie (z możliwie tym samym tłem i oświetleniem, niekoniecznie z tym samym wyrazem twarzy) (jeżeli mieszkamy sami - robimy je tylko sobie, ale wykorzystujemy talent aktorski i na 15+ jesteśmy radośni, a na 15+ przygnębieni, etc.). Generalnie im większy i bogatszy zbiór, tym ciekawsze będą wyniki. Warto też zadbać, by twarze na zdjęciach były jak najlepiej wyśrodkowane (nie muszą być wyśrodkowane idealnie), patrzyły do przodu i miały możliwie jednorodne tło (oczywiście to nigdy nie uda się idealnie - proszę zwrócić uwagę jak drobne niedoskonałości wpłyną na wyniki). Przy tworzeniu zbioru można też współpracować z innymi studentami z tego kursu. ;]

W celu oszczędności na danych osobowych i czasie, korzystam z istniejącego datasetu twarzy:

In [ ]:
faces_ds_name = "olivettifaces.mat"

In [ ]:
if not os.path.exists(faces_ds_name):
    !wget https://cs.nyu.edu/~roweis/data/olivettifaces.mat

faces_ds = loadmat("olivettifaces.mat")["faces"]
faces = np.array([
    faces_ds[:, i].reshape(64, 64).T
    for i in range(faces_ds.shape[1])
])

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(16):
    plt.subplot(4,4, 1 + i)
    plt.imshow(faces[10*i], cmap="gray")
fig.suptitle("Przykładowe twarze z datasetu")
plt.show()

Konwertujemy je do czerni i bieli oraz zmniejszamy ich rozdzielczość (jak silnie? tak, aby obliczenia nie trwały zbyt długo). Traktujemy je jako zbiór n*(15+) elementów należących do n klas. Dokonujemy zamiany zdjęć na wektory i przeprowadzamy na takim zbiorze PCA (zwykłe, nie kernel). 

In [ ]:
identity_pca = PCA()
faces_flat = faces.reshape(faces.shape[0], -1)
identity_pca.fit(faces_flat)

Jak wygląda "średnia twarz"?

In [ ]:
plt.imshow(faces.mean(axis=0), cmap="gray")
plt.title("Średnia twarz")
plt.show()

### Komentarz
Średnia twarz w istocie wygląda jak rozmazana twarz.

Jak wyglądają kolejne "principal components" (są bardzo długimi wektorami, ale możemy je przekonwertować ponownie na zdjęcia i tak zaprezentować)? 

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle("Wizualizacja principal components zbioru")
for i, component in enumerate(identity_pca.components_[:16]):
    plt.subplot(4, 4, 1 + i)
    plt.imshow(component.reshape(64, 64), cmap="gray")
plt.show()

Zauważmy też, że tylko niewielka część nowych wektorów bazowych ma istotny udział w tłumaczeniu pierwotnej wariancji zbioru (explained variance ratio). 

In [ ]:
plt.xlim(-15, 0)
plt.title("Histogram variance ratio wektorów bazowych")
plt.hist(
    np.log(identity_pca.explained_variance_ratio_), 
    bins=identity_pca.explained_variance_ratio_.shape[0]
)
plt.xlabel("Rząd wielkości variance ratio")
plt.ylabel("Liczba wektorów bazowych o danym variance ratio")
plt.show()

### Komentarz
Powyżej widzimy histogram logarytmów (a więc rzędów wielkości) udziałów wektorów bazowych w datasecie. Największe udziały osiągają rząd wielkości $10^{-1}$, natomiast większość ma rzędy wielkości ok. $10^{-8}$


Jak wyglądają zrekonstruowane twarze, jeżeli obetniemy przestrzeń do 5, 15 i 50 najważniejszych? 

In [ ]:
pcas = [identity_pca] + [PCA(n_components=n).fit(faces_flat) for n in [2, 5,10,15, 20]]
inverse_flat = [
    pca.inverse_transform(pca.transform(faces_flat)) 
    for pca in pcas
]

In [ ]:
for i in[2, 100, 16, 32]:
    fig = plt.figure(figsize=(10,20))
    for j, pca in enumerate(pcas):
        plt.subplot(1, len(pcas), j+1)
        plt.title(f"n={pca.n_components_}")
        face = inverse_flat[j][i].reshape(64, 64)
        plt.imshow(face, cmap="gray") 
        
plt.show()

Na koniec, ograniczmy przestrzeń do 2 najważniejszych wymiarów i zrzutujmy elementy zbioru na płaszczyznę 2D (kolorując punkty w zależności od klasy do której przynależą). Czy są łatwo separowalne?

In [ ]:
two_pca = PCA(n_components=2).fit(faces_flat)
inv_flat = two_pca.transform(faces_flat) 
classes = [(i//10) for i in range(inv_flat.shape[0])]
# korzystam z tego, że n-ta osoba jest na zdjęciach o numerach (10*n, 10*n + 9)

In [ ]:
colors_list = list(matplotlib.colors.get_named_colors_mapping().keys())
np.random.seed(0)
np.random.shuffle(colors_list)

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle("Rozrzut skonwertowanych do 2 wektorów bazowych twarzy osób o danym numerze")
for c in np.unique(classes)[:10]:
    plt.scatter(
        inv_flat[10*c:10*c+9, 0], 
        inv_flat[10*c:10*c+9, 1], 
        c=colors_list[c], 
        marker=f"${c}$",
        s=300
    )
plt.show()

### Komentarz
Można zauważyć, że żadna z klas nie jest zupełnie losowo rozrzucona po przestrzeni wyznaczonej przez dwa wektory bazowe okreslone przez PCA - zajmują one pewne konkretne obszary. Natomiast nie da się powiedziec, że te obszary są łatwo separowalne.